# Bridge Detection

In [34]:
%load_ext autoreload
%autoreload 2

import json
import os
import sys
sys.path.insert(0, '/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset')
import header as head
import bridge_header as bh

bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"
follow_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow"
react_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/react"
FR_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. Nodes that have followers from both communities. 
2. Nodes that follows other communites but only has follower from one commmunity

**code preparation**

In [ ]:
# Obtained who followed by whom relationship
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

who_followed_by_whom = {}
# Find who followed by whom
for user in follows:
    who_followed_by_whom[user] = []
    followed_by = who_followed_by_whom[user]

    for f in follows:
        if user in follows[f]: followed_by.append(f)

# head.writeToJSON(head.getRoot(),"who_followed_by_whom",who_followed_by_whom)

In [ ]:
# obtain all followers from both communities per user 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as f:
    communities = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_followed_by_whom.json") as f:
    followed_by = json.load(f)

rumours_set = communities["rumours"]
non_rumours_set = communities["non_rumours"]
uncategorized_set = communities["uncategorized"]

bridges = {}
ids = []
for c in communities.keys():
    ids.extend(communities[c])


for id in ids:
    try:
        followers = followed_by[id]
    except: 
        continue
    
    bridges[id] = {
        "rumours" : [],
        "non_rumours" : [],
        "uncategorized" : []
    }
    node = bridges[id]

    for followee in followers:
        if followee in rumours_set: node["rumours"].append(followee)
        elif followee in non_rumours_set: node["non_rumours"].append(followee)
        elif followee in uncategorized_set: node["uncategorized"].append(followee)
    
    # uncomment the following if statement to obtain "user_followed_by_unfiltered" file
    if (len(node["rumours"])==0 and len(node["non_rumours"])==0
        and len(node["uncategorized"])==0): bridges.pop(id)

head.writeToJSON(bridges_path,"user_followed_by",bridges)

We test whether type 3 bridges affect the identification of type 1 bridge. 

Result: type 3 bridges does not affect the amount of type 1 bridges being identified

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/bridges.json") as file:
    b1 = json.load(file)
    b1 = b1.keys()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/bridges_v2.json") as file:
    b2 = json.load(file)
    b2 = b2.keys()

print(b1==b2)

Update user_followed_by with additional information about bridges

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as file:
    followed_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/bridges.json") as file:
    bridges = json.load(file)
    bridge_ids = bridges.keys()

new_followed_by = {}

for f in followed_by:
    user = followed_by[f]
    new_followed_by[f] = {
        "rumours":[],
        "non_rumours":[],
        "uncategorized":[],
        "bridges":[]
    }

    for id in user["rumours"]:
        if id in bridge_ids : (new_followed_by[f])["bridges"].append(id)
        else : (new_followed_by[f])["rumours"].append(id)
    
    for id in user["non_rumours"]:
        if id in bridge_ids : (new_followed_by[f])["bridges"].append(id)
        else : (new_followed_by[f])["non_rumours"].append(id)

    for id in user["uncategorized"]:
        if id in bridge_ids : (new_followed_by[f])["bridges"].append(id)
        else : (new_followed_by[f])["uncategorized"].append(id)

head.writeToJSON(bridges_path,"user_followed_by",new_followed_by)

## Type 1 Bridge Analysis

### Follow

In [9]:
# Get type 1 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/user_followed_by.json") as file:
    followed_by = json.load(file)

head.writeToJSON(head.makePath([follow_path,"type 1"]),"type1",bh.getType1(followed_by))

In [3]:
# Get who type 1 bridges follow
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/type 1/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    follows = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow/with_bridges.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([follow_path,"type 1"]),"follow_whom",bh.getFollows(type1,follows,communities))

In [13]:
# Which community does each type 1 bridge originate from
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/type 1/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([follow_path,"type 1"]),"originate_from",bh.originateFrom(type1,communities))

In [26]:
# What kind of tweets does each type 1 bridge tweets
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/type 1/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/tweets_type_by_userId.json") as file:
    tweets_type = json.load(file)

head.writeToJSON(head.makePath([follow_path,"type 1"]),"tweets_type",bh.tweets_type(type1,tweets_type))

### Follow+React

In [41]:
# Get who type 1 bridges follow
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type 1/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/FR_to_whom.json") as file:
    RF_relations = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow+react/communities.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([FR_path,"type 1"]),"FR_whom",bh.getFollows(type1,RF_relations,communities))

In [6]:
# Which community does each type 1 bridge originate from
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([FR_path,"type 1"]),"originate_from",bh.originateFrom(type1,communities))

In [7]:
# What kind of tweets does each type 1 bridge tweets
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/tweets_type_by_userId.json") as file:
    tweets_type = json.load(file)

head.writeToJSON(head.makePath([FR_path,"type 1"]),"tweets_type",bh.tweets_type(type1,tweets_type))

Obtaining centrality information regarding type 1 bridges' followers

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/betweeness/directed_betweenness.json") as file:
    betweeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/closeness/rumour_closeness.json") as file:
    rum_closeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/closeness/non_rumour_closeness.json") as file:
    nonrum_closeness = json.load(file)

type1_centrality = {}

for id in type1:
    bridge = type1[id]
    type1_centrality[id] = {
        "rum_count" : len((bridge["followed_by"])["rumours"]),
        "nonrum_count" : len((bridge["followed_by"])["non_rumours"]),
        "rumours" : {},
        "non_rumours" : {},
    }

    rum_followers = (type1_centrality[id])["rumours"]
    nonrum_followers = (type1_centrality[id])["non_rumours"]

    for user in (bridge["followed_by"])["rumours"]:
        betw = (betweeness["rumours"])[user]
        close = (rum_closeness["rumours"])[user]
        if betw > 0 or close > 0:
            rum_followers[user] = {
                "betweeness":(betweeness["rumours"])[user],
                "closeness":(rum_closeness["rumours"])[user]
            }
    
    for user in (bridge["followed_by"])["non_rumours"]:
        betw = (betweeness["non_rumours"])[user]
        close = (nonrum_closeness["non_rumours"])[user]
        if betw > 0 or close > 0:
            nonrum_followers[user] = {
                "betweeness":(betweeness["non_rumours"])[user],
                "closeness":(nonrum_closeness["non_rumours"])[user]
            }

head.writeToJSON(head.makePath([bridges_path,"type 1"]),"centrality_2",type1_centrality)

### Follow+React

In [3]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/betweeness/betweeness.json") as file:
    betweeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/closeness/nonrumours.json") as file:
    nonrum_closeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/closeness/rumours.json") as file:
    rum_closeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/closeness/nonrumours+uncategorized.json") as file:
    uncat_nonrum_closeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/closeness/rumours+uncategorized.json") as file:
    uncat_rum_closeness = json.load(file)

centralities = bh.get_centralities(type1,betweeness,nonrum_closeness,rum_closeness,uncat_nonrum_closeness,uncat_rum_closeness)
head.writeToJSON(head.makePath([bridges_path,"follow+react"]),"centralities",centralities)

## Type 2 Bridge Analysis

### Follow

In [27]:
# Get type 2 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/user_followed_by.json") as file:
    followed_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    follows = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow/with_bridges.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([follow_path,"type 2"]),"type2_2",bh.getType2(followed_by,follows,communities))

### Follow+React

In [31]:
# Get type 2 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/react+following.json") as file:
    FR_relations = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    follows = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow+react/communities.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([FR_path,"type 2"]),"type2",bh.getType2(FR_relations,follows,communities))

## Type 3 Bridge Analysis

### Follow

In [35]:
# Get type 3 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/user_followed_by.json") as file:
    followed_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow/with_bridges.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([follow_path,"type 3"]),"type 3",bh.getType3(followed_by,communities))

### Follow+React

In [36]:
# Get type 3 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/react+following.json") as file:
    FR_relations = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow+react/communities.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([FR_path,"type 3"]),"type 3",bh.getType3(FR_relations,communities))

In [40]:
# Get who type 3 bridges follow
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type 3/type 3.json") as file:
    type3 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/FR_to_whom.json") as file:
    RF_relations = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow+react/communities.json") as file:
    communities = json.load(file)

head.writeToJSON(head.makePath([FR_path,"type 3"]),"FR_whom",bh.getFollows(type3,RF_relations,communities))